In [38]:
import torch
from models.resnet_simclr import ResNetSimCLR

arch = 'resnet18'
out_dim = 128
model = ResNetSimCLR(arch, out_dim)
model.load_state_dict(torch.load('./runs/Oct26_14-21-38_ems1/checkpoint_0100.pth.tar')['state_dict'])
model.eval()

ResNetSimCLR(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [39]:
from data_aug.contrastive_learning_dataset import SimCLRCustom
import torchvision.transforms as transforms

root_folder = '../crabs_dataset/image_matching/test'
test_transform = transforms.Compose([transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(0.5, 0.5)])

infer_dataset = SimCLRCustom(root_folder, transform=test_transform)

In [40]:
inputs_A, target_A = infer_dataset[0]
inputs_B, target_B = infer_dataset[1]

input_A_1, input_A_2 = inputs_A
input_B_1, input_B_2 = inputs_B

input_A_1 = torch.unsqueeze(input_A_1, dim=0)
input_A_2 = torch.unsqueeze(input_A_2, dim=0)
input_B_1 = torch.unsqueeze(input_B_1, dim=0)
input_B_2 = torch.unsqueeze(input_B_2, dim=0)

In [44]:
import torch.nn.functional as F
feature_A_1 = model(input_A_1)
feature_A_2 = model(input_A_2)
feature_B_1 = model(input_B_1)
feature_B_2 = model(input_B_2)

In [54]:

score1 = F.cosine_similarity(feature_A_1, feature_A_1)
score2 = F.cosine_similarity(feature_A_1, feature_A_2)
score3 = F.cosine_similarity(feature_A_1, feature_B_1)
score4 = F.cosine_similarity(feature_A_1, feature_B_2)

print(score1)
print(score2)
print(score3)
print(score4)

tensor([1.], grad_fn=<SumBackward1>)
tensor([1.], grad_fn=<SumBackward1>)
tensor([0.0291], grad_fn=<SumBackward1>)
tensor([-0.0227], grad_fn=<SumBackward1>)


In [60]:
from sklearn.metrics.pairwise import cosine_similarity     
   

feature_A = torch.cat([feature_A_1,feature_A_2,feature_B_1,feature_B_2])
feature_B = torch.cat([feature_A_1,feature_A_2,feature_B_1,feature_B_2])
feature_A = feature_A.cpu().detach().numpy()     
feature_B = feature_B.cpu().detach().numpy()     

print(feature_A.shape)
print(feature_B.shape)
cosine_similarity(feature_A, feature_B)


(4, 128)
(4, 128)


array([[ 1.        ,  1.        ,  0.02911378, -0.0226875 ],
       [ 1.        ,  1.        ,  0.02911378, -0.0226875 ],
       [ 0.02911378,  0.02911378,  1.0000001 ,  0.82091105],
       [-0.0226875 , -0.0226875 ,  0.82091105,  0.99999964]],
      dtype=float32)

In [47]:
input1 = torch.randn(100, 128)
input2 = torch.randn(1, 128)
output = F.cosine_similarity(input1, input2)
print(output.shape)


torch.Size([100])
